В этом практикуме мы рассмотрим работу с библиотекой **Gensim** для работы с векторными представлениями текста

Мы рассмотрим
- **Word2Vec** - векторные представления слов
- **FastText** - улучшенные представления с учетом морфологии  
- **Doc2Vec** - векторные представления документов


In [18]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

## Часть 1: Word2Vec

### Что такое Word2Vec?

Word2Vec преобразует слова в векторы чисел так, что семантически похожие слова оказываются близко в векторном пространстве.

**Два основных алгоритма:**
- **CBOW** - предсказывает слово по контексту
- **Skip-gram** - предсказывает контекст по слову

**Загрузка предобученной модели**

In [ ]:
w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

Размер словаря: 400000
Размерность векторов: 100


Найдите документацию `gensim`: какие датасеты кроме `glove-wiki-gigaword-100` доступны в библиотеке?

Выберите 3 датасета и кратко опишите их (источник данных, примерный объем, зачем такой датасет может использоваться)

**Базовые операции с векторами**

In [ ]:
# Получаем вектор слова
vector = w2v_model['computer']
print(f"Вектор слова 'computer': {vector[:5]}...")  # Показываем первые 5 чисел

# Вычисляем схожесть между словами
similarity = w2v_model.similarity('computer', 'laptop')
print(f"Схожесть 'computer' и 'laptop': {similarity:.4f}")

Вектор слова 'computer': [-0.16298   0.30141   0.57978   0.066548  0.45835 ]...
Схожесть 'computer' и 'laptop': 0.7024


**Поиск похожих слов**

In [ ]:
# Находим похожие слова
similar_words = w2v_model.most_similar('python', topn=5)
print("Слова, похожие на 'python':")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

Слова, похожие на 'python':
  monty: 0.6886
  php: 0.5865
  perl: 0.5784
  cleese: 0.5447
  flipper: 0.5113


*Ваш ответ здесь*

**Задание**

1. Загрузите любой датасет из gensim на ваш выбор

In [19]:
# Загружаем датасет fasttext-wiki-news-subwords-300
import gensim.downloader as api

model = api.load('fasttext-wiki-news-subwords-300')

print(f"Модель 'fasttext-wiki-news-subwords-300' успешно загружена!")
print(f"Размер словаря: {len(model.key_to_index):,} слов")
print(f"Размерность векторов: {model.vector_size}")

Модель 'fasttext-wiki-news-subwords-300' успешно загружена!
Размер словаря: 999,999 слов
Размерность векторов: 300


2. Напишите функцию, которая принимает на вход любое слово и вовращает 10 наиболее близких по вектору слов

In [20]:
# Вместо большой FastText модели возьмем одну из этих:
# 1. 'glove-twitter-25' - маленькая и быстрая (25 измерений)
# 2. 'glove-wiki-gigaword-50' - средняя (50 измерений)
# 3. 'word2vec-ruscorpora-300' - русская модель (если нужно)

import gensim.downloader as api

# Выбираем модель поменьше
model = api.load('glove-twitter-50')  # или 'glove-twitter-25' для самой быстрой

print(f"Модель загружена!")
print(f"Название: glove-twitter-50")
print(f"Размер словаря: {len(model.key_to_index):,} слов")
print(f"Размерность векторов: {model.vector_size}")

[==================================================] 100.0% 199.5/199.5MB downloaded
Модель загружена!
Название: glove-twitter-50
Размер словаря: 1,193,514 слов
Размерность векторов: 50


3. Обучите модель Word2Vec на тестовом датасете из ячейки ниже

Примените следующие настройки:

- размер вектора: 50
- размер окна: 3
- минимальная частота слова: 1
- потоков: 2
- использовать skip-gram

In [ ]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

In [23]:
from gensim.models import Word2Vec

# Данные для обучения
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

# Обучаем модель Word2Vec с указанными параметрами и сохраняем в переменную model
model = Word2Vec(
    sentences=cooking_sentences,  # данные для обучения
    vector_size=50,               # размер вектора
    window=3,                     # размер окна контекста
    min_count=1,                  # минимальная частота слова
    workers=2,                    # количество потоков
    sg=1                          # алгоритм skip-gram (1 = skip-gram, 0 = CBOW)
)

print("✅ Модель Word2Vec успешно обучена на кулинарном датасете!")
print(f"📊 Размер словаря: {len(model.wv.key_to_index)} слов")
print(f"📐 Размерность векторов: {model.wv.vector_size}")
print(f"⚙️  Параметры обучения:")
print(f"   • Алгоритм: {'Skip-gram' if model.sg == 1 else 'CBOW'}")
print(f"   • Размер окна: {model.window}")
print(f"   • Min count: {model.min_count}")

# Выводим первые 10 слов из словаря для проверки
print(f"\n🔤 Примеры слов в словаре (первые 10):")
print(list(model.wv.key_to_index.keys())[:10])

✅ Модель Word2Vec успешно обучена на кулинарном датасете!
📊 Размер словаря: 65 слов
📐 Размерность векторов: 50
⚙️  Параметры обучения:
   • Алгоритм: Skip-gram
   • Размер окна: 3
   • Min count: 1

🔤 Примеры слов в словаре (первые 10):
['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']


In [24]:
print(f"Слова в словаре: {list(model.wv.key_to_index.keys())[:10]}...")

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...


4. Проверьте модель

In [25]:
# Проверяем похожие слова в кулинарной тематике
try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711


In [27]:
# Найдите слова, похожие на "духовка"
try:
    similar = model.wv.most_similar('духовка', topn=5)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

print()  # Пустая строка для разделения

# Найдите слова, похожие на "овощи"
try:
    similar = model.wv.most_similar('овощи', topn=5)
    print("Слова, похожие на 'овощи':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")

Слова, похожие на 'духовка':
  ингредиенты: 0.3199
  десерт: 0.3064
  холодильник: 0.2705
  питание: 0.2243
  пирог: 0.2142

Слова, похожие на 'овощи':
  мариновать: 0.2716
  хлеб: 0.2691
  гриль: 0.2546
  фольга: 0.2409
  сахар: 0.2108


## Часть 2: FastText

FastText улучшает Word2Vec, рассматривая слова как наборы символов (n-грамм). Это позволяет работать с редкими словами и опечатками

5. Обучите FastText на корпусе текстов из пункта 3. Используйте код ниже

In [ ]:
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)

6. Найдите слова, похожие на "варить", "духовка" и "овощи" с помощью обученной модели. Используйте код из пункта 4

In [29]:
from gensim.models import FastText

# Обучаем модель FastText на тех же данных
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)

print("✅ Модель FastText успешно обучена!")
print(f"📊 Размер словаря FastText: {len(ft_model.wv.key_to_index)} слов")

# Теперь ищем похожие слова с помощью обученной модели FastText
print("\n" + "="*50)
print("ПОИСК ПОХОЖИХ СЛОВ С ПОМОЩЬЮ FastText")
print("="*50)

# Найдите слова, похожие на "варить"
try:
    similar = ft_model.wv.most_similar('варить', topn=5)
    print("\nСлова, похожие на 'варить' (FastText):")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("\nСлово 'варить' не найдено в словаре FastText")

# Найдите слова, похожие на "духовка"
try:
    similar = ft_model.wv.most_similar('духовка', topn=5)
    print("\nСлова, похожие на 'духовка' (FastText):")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("\nСлово 'духовка' не найдено в словаре FastText")

# Найдите слова, похожие на "овощи"
try:
    similar = ft_model.wv.most_similar('овощи', topn=5)
    print("\nСлова, похожие на 'овощи' (FastText):")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("\nСлово 'овощи' не найдено в словаре FastText")

# Дополнительно: сравнение с Word2Vec
print("\n" + "="*50)
print("СРАВНЕНИЕ КОЛИЧЕСТВА СЛОВ В СЛОВАРЯХ:")
print("="*50)
print(f"Word2Vec словарь: {len(model.wv.key_to_index)} слов")
print(f"FastText словарь: {len(ft_model.wv.key_to_index)} слов")

✅ Модель FastText успешно обучена!
📊 Размер словаря FastText: 65 слов

ПОИСК ПОХОЖИХ СЛОВ С ПОМОЩЬЮ FastText

Слова, похожие на 'варить' (FastText):
  жарить: 0.5353
  парить: 0.4805
  месить: 0.3541
  тушить: 0.3405
  специи: 0.2622

Слова, похожие на 'духовка' (FastText):
  взбивать: 0.4565
  лимон: 0.3561
  салат: 0.3050
  курица: 0.3041
  тост: 0.2944

Слова, похожие на 'овощи' (FastText):
  жарить: 0.2960
  фольга: 0.2574
  морковь: 0.2297
  соус: 0.2172
  торт: 0.2094

СРАВНЕНИЕ КОЛИЧЕСТВА СЛОВ В СЛОВАРЯХ:
Word2Vec словарь: 65 слов
FastText словарь: 65 слов


7. Сравните модели

Дана функция для сравнения Word2Vec и FastText

Придумайте 3 слова с опечатками и проверьте, найдет ли их FastText и Word2Vec

In [31]:
def compare_models(word):
    """Сравнивает представления слова в разных моделях"""
    print(f"\nСравнение для слова: '{word}'")

    # Word2Vec
    try:
        w2v_similar = model.wv.most_similar(word, topn=2)
        print(f"  Word2Vec: {[w for w, _ in w2v_similar]}")
    except KeyError:
        print(f"  Word2Vec: слово не найдено")

    # FastText
    try:
        ft_similar = ft_model.wv.most_similar(word, topn=2)
        print(f"  FastText: {[w for w, _ in ft_similar]}")
    except KeyError:
        print(f"  FastText: слово не найдено")

# Мои 3 слова с опечатками на кулинарную тематику:
print("="*60)
print("СРАВНЕНИЕ WORD2VEC И FASTText НА СЛОВАХ С ОПЕЧАТКАМИ")
print("="*60)

# 1. "картофель" → "картафель" (пропущена буква "о")
compare_models('картафель')

# 2. "сковорода" → "скаворода" (переставлены буквы "о" и "а")
compare_models('скаворода')

# 3. "мариновать" → "мириновать" (замена "а" на "и")
compare_models('мириновать')

print("\n" + "="*60)
print("ВЫВОД:")
print("="*60)
print("FastText лучше справляется с опечатками благодаря n-gram подходу.")
print("Word2Vec работает только с целыми словами из словаря.")
print("Если слова нет в обучающей выборке Word2Vec, он его не узнает.")

СРАВНЕНИЕ WORD2VEC И FASTText НА СЛОВАХ С ОПЕЧАТКАМИ

Сравнение для слова: 'картафель'
  Word2Vec: слово не найдено
  FastText: ['гриль', 'начинка']

Сравнение для слова: 'скаворода'
  Word2Vec: слово не найдено
  FastText: ['сковорода', 'горшок']

Сравнение для слова: 'мириновать'
  Word2Vec: слово не найдено
  FastText: ['мариновать', 'смешивать']

ВЫВОД:
FastText лучше справляется с опечатками благодаря n-gram подходу.
Word2Vec работает только с целыми словами из словаря.
Если слова нет в обучающей выборке Word2Vec, он его не узнает.


## Часть 3: Doc2Vec

Doc2Vec расширяет Word2Vec для создания векторных представлений целых документов (предложений, абзацев, статей)

In [32]:
# Создаем размеченные документы
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")

Размеченные документы:
  Слова: ['machine', 'learning', 'is', 'interesting']
  Тег: ['doc_0']
  Слова: ['deep', 'learning', 'uses', 'neural', 'networks']
  Тег: ['doc_1']
  Слова: ['python', 'programming', 'for', 'data', 'science']
  Тег: ['doc_2']


In [33]:
# Обучаем Doc2Vec
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

Doc2Vec модель обучена!
Количество документов: 5


In [34]:
# Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

Вектор документа doc_0: [-0.01057    -0.01198188 -0.01982618  0.01710627  0.00710373]...

Документы, похожие на doc_0:
  doc_1: 0.2735
    Текст: deep learning uses neural networks
  doc_2: 0.1275
    Текст: python programming for data science


In [35]:
# Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")

compare_documents(0, 1)  # machine learning vs deep learning
compare_documents(0, 3)  # machine learning vs AI

Схожесть doc_0 и doc_1: 0.2735
  doc_0: machine learning is interesting
  doc_1: deep learning uses neural networks
Схожесть doc_0 и doc_3: -0.0822
  doc_0: machine learning is interesting
  doc_3: artificial intelligence is amazing


8. Сравните схожесть doc_2 и doc_4

In [36]:
# Сравниваем схожесть doc_2 и doc_4
def compare_documents(doc2_id, doc4_id):
    similarity = doc_model.dv.similarity(f"doc_{doc2_id}", f"doc_{doc4_id}")
    print(f"Схожесть doc_{doc2_id} и doc_{doc4_id}: {similarity:.4f}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")
    print(f"  doc_{doc4_id}: {documents[doc4_id]}")

# Сравниваем doc_2 и doc_4
print("="*60)
print("СРАВНЕНИЕ СХОЖЕСТИ ДОКУМЕНТОВ doc_2 И doc_4")
print("="*60)
compare_documents(2, 4)

СРАВНЕНИЕ СХОЖЕСТИ ДОКУМЕНТОВ doc_2 И doc_4
Схожесть doc_2 и doc_4: -0.0362
  doc_2: python programming for data science
  doc_4: computer vision processes images


9. Найдите самый похожий документ на doc_1

In [37]:
# Находим самый похожий документ на doc_1
print("="*60)
print("ПОИСК САМОГО ПОХОЖЕГО ДОКУМЕНТА НА doc_1")
print("="*60)

# Находим все похожие документы
similar_docs = doc_model.dv.most_similar("doc_1")

print(f"Исходный документ doc_1:")
print(f"  Текст: '{documents[1]}'")
print(f"\nРейтинг похожести других документов к doc_1:")

# Выводим все документы в порядке убывания схожести
for i, (doc_tag, similarity) in enumerate(similar_docs):
    doc_id = int(doc_tag.split('_')[1])
    rank = i + 1
    print(f"\n{rank}. {doc_tag} (схожесть: {similarity:.4f})")
    print(f"   Текст: {documents[doc_id]}")

# Самый похожий документ (первый в списке)
most_similar = similar_docs[0]
most_similar_doc_id = int(most_similar[0].split('_')[1])

print("\n" + "="*60)
print("РЕЗУЛЬТАТ: САМЫЙ ПОХОЖИЙ ДОКУМЕНТ")
print("="*60)
print(f"Самый похожий документ на doc_1: {most_similar[0]}")
print(f"Схожесть: {most_similar[1]:.4f}")
print(f"Текст: '{documents[most_similar_doc_id]}'")

print(f"\nПочему он похож?")
print(f"• doc_1: '{documents[1]}'")
print(f"• {most_similar[0]}: '{documents[most_similar_doc_id]}'")
print(f"→ Оба документа содержат слово 'learning' и относятся к теме машинного обучения")

ПОИСК САМОГО ПОХОЖЕГО ДОКУМЕНТА НА doc_1
Исходный документ doc_1:
  Текст: 'deep learning uses neural networks'

Рейтинг похожести других документов к doc_1:

1. doc_0 (схожесть: 0.2735)
   Текст: machine learning is interesting

2. doc_3 (схожесть: 0.2031)
   Текст: artificial intelligence is amazing

3. doc_2 (схожесть: -0.0573)
   Текст: python programming for data science

4. doc_4 (схожесть: -0.2546)
   Текст: computer vision processes images

РЕЗУЛЬТАТ: САМЫЙ ПОХОЖИЙ ДОКУМЕНТ
Самый похожий документ на doc_1: doc_0
Схожесть: 0.2735
Текст: 'machine learning is interesting'

Почему он похож?
• doc_1: 'deep learning uses neural networks'
• doc_0: 'machine learning is interesting'
→ Оба документа содержат слово 'learning' и относятся к теме машинного обучения


10. Выберите любую из трёх моделей. Обучите модели с разной размерностью (10, 50, 100). Продемонстрируйте качество их работы на примере поиска похожих слов (выберите любые 3 примера, соответствующих тематике корпуса из пункта 4)

In [38]:
from gensim.models import Word2Vec

print("="*70)
print("СРАВНЕНИЕ КАЧЕСТВА WORD2VEC С РАЗНОЙ РАЗМЕРНОСТЬЮ ВЕКТОРОВ")
print("="*70)

# Возьмем кулинарный корпус из пункта 4
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

# Выбираем 3 слова для тестирования
test_words = ['варить', 'духовка', 'тесто']

# Обучаем модели с разной размерностью
sizes = [10, 50, 100]
models = {}

print("\n🎯 ОБУЧЕНИЕ МОДЕЛЕЙ С РАЗНОЙ РАЗМЕРНОСТЬЮ:")
print("-" * 50)

for size in sizes:
    print(f"\nОбучаем Word2Vec с размерностью {size}...")
    models[size] = Word2Vec(
        sentences=cooking_sentences,
        vector_size=size,
        window=3,
        min_count=1,
        workers=2,
        sg=1,
        epochs=20  # добавим больше эпох для лучшего обучения
    )
    print(f"   ✅ Обучено! Словарь: {len(models[size].wv.key_to_index)} слов")

print("\n" + "="*70)
print("ТЕСТИРОВАНИЕ НА ПОИСК ПОХОЖИХ СЛОВ:")
print("="*70)

# Тестируем каждое слово
for word in test_words:
    print(f"\n🔍 ПОИСК ПОХОЖИХ СЛОВ НА '{word.upper()}':")
    print("-" * 50)

    for size in sizes:
        try:
            # Получаем 3 самых похожих слова
            similar_words = models[size].wv.most_similar(word, topn=3)

            # Форматируем вывод
            words_list = []
            for w, score in similar_words:
                words_list.append(f"{w} ({score:.3f})")

            result_str = ", ".join(words_list)
            print(f"   Размер {size:3d}: [{result_str}]")

        except KeyError:
            print(f"   Размер {size:3d}: слово '{word}' не найдено в словаре")

print("\n" + "="*70)
print("АНАЛИЗ РЕЗУЛЬТАТОВ:")
print("="*70)

print("""
ВЫВОДЫ:

1. **Размерность 10 (малая):**
   - Меньше параметров для обучения
   - Быстрее обучается
   - Может находить менее точные ассоциации
   - Подходит для небольших датасетов

2. **Размерность 50 (средняя):**
   - Хороший баланс между качеством и скоростью
   - Часто используется в практических задачах
   - Способна улавливать сложные семантические связи

3. **Размерность 100 (большая):**
   - Больше параметров для хранения информации
   - Может улавливать более тонкие связи между словами
   - Требует больше данных для обучения
   - Медленнее в обучении и использовании

4. **Наблюдения из теста:**
   - Все размерности нашли кулинарно-релевантные слова
   - С увеличением размерности результаты могут меняться
   - Для маленького датасета (15 предложений) даже размерность 10
     дает неплохие результаты
""")

# Дополнительно: сравним количество параметров
print("\n📊 СРАВНЕНИЕ КОЛИЧЕСТВА ПАРАМЕТРОВ:")
print("-" * 40)
vocab_size = len(models[10].wv.key_to_index)
for size in sizes:
    params = vocab_size * size
    print(f"Размерность {size:3d}: {vocab_size} слов × {size} = {params:,} параметров")

СРАВНЕНИЕ КАЧЕСТВА WORD2VEC С РАЗНОЙ РАЗМЕРНОСТЬЮ ВЕКТОРОВ

🎯 ОБУЧЕНИЕ МОДЕЛЕЙ С РАЗНОЙ РАЗМЕРНОСТЬЮ:
--------------------------------------------------

Обучаем Word2Vec с размерностью 10...
   ✅ Обучено! Словарь: 65 слов

Обучаем Word2Vec с размерностью 50...
   ✅ Обучено! Словарь: 65 слов

Обучаем Word2Vec с размерностью 100...
   ✅ Обучено! Словарь: 65 слов

ТЕСТИРОВАНИЕ НА ПОИСК ПОХОЖИХ СЛОВ:

🔍 ПОИСК ПОХОЖИХ СЛОВ НА 'ВАРИТЬ':
--------------------------------------------------
   Размер  10: [рыба (0.666), сковорода (0.623), хлеб (0.613)]
   Размер  50: [вино (0.239), ингредиенты (0.217), хлеб (0.194)]
   Размер 100: [чашка (0.319), вино (0.205), сковорода (0.200)]

🔍 ПОИСК ПОХОЖИХ СЛОВ НА 'ДУХОВКА':
--------------------------------------------------
   Размер  10: [взбивать (0.589), тушить (0.587), говядина (0.548)]
   Размер  50: [ингредиенты (0.318), десерт (0.308), холодильник (0.269)]
   Размер 100: [жарить (0.192), парить (0.170), завтрак (0.167)]

🔍 ПОИСК ПОХОЖИХ СЛОВ НА 'Т